In [1]:
from ngsolve import *
from math import pi
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
# Draw(mesh)
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")

u,v = fes.TnT()

time = 0.0
dt = 0.001

In [3]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
# Draw(b,mesh,"wind")
from ngsolve.internal import visoptions
visoptions.scalfunction = "wind:0"

In [4]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

In [5]:
gfu = GridFunction(fes)

In [6]:
t = Parameter(0.0)

In [7]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
# scene = Draw(gausspt,mesh,"ft",order=3)
# time = 0.0
# from time import sleep
# while time < 10 - 0.5 * dt:
#     t.Set(time)
#     scene.Redraw()
#     time += 1e-3
#     sleep(1e-6)

In [8]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

In [9]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p

In [10]:
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

In [11]:
butchertab = sdirk5()   
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

In [12]:
ft = LinearForm(fes)
ft += gausspt*v*dx
# uD = ((1-y*y)*x)
time = 0.0
t.Set(0.0)
gfu.Set(((1-y*y)*x))
scene = Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [13]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    ft.Assemble()
    Mu0.data = m.mat * gfu.vec + dt*ft.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(((1-y*y)*x))
        ui.data = gfu.vec; rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        # compute k[i] from ui
        k[i].data = - a.mat * ui
    t_intermediate += dt; t.Set(time+t_intermediate)
    # Adding up the ks:
    gfu.Set(((1-y*y)*x))
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0        
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print(""); time+=t_intermediate  

 9.999999999999897676
